In [1]:
import IPython.display as ipd
import os
import librosa as alib
import pandas as pd
from sklearn.neural_network import MLPClassifier

In [2]:
ipd.Audio('emotion/Actor_01/03-02-01-01-01-01-01.wav')

In [3]:
emotions = {'01':'neutral', '02' :'calm', '03' : 'happy', '04' : 'sad', '05':'angry', '06' : 'fearful', '07' : 'disgust', '08' : 'surprised'}

In [4]:
directory = 'emotion/'


In [5]:
def extract_features(path):
        sig,rate = alib.load(path)
        mfcc = alib.feature.mfcc(sig,rate,n_mfcc=30)
        ch_stft = alib.feature.chroma_stft(sig,rate)
        chroma_stft_mean = ch_stft.mean(axis=1).tolist()
        chroma_stft_var = ch_stft.var(axis=1).tolist()
        mfcc_mean = mfcc.mean(axis=1).tolist()
        mfcc_var = mfcc.var(axis=1).tolist()
        return chroma_stft_mean+chroma_stft_var+mfcc_mean+mfcc_var

In [6]:
import datetime as dt

In [7]:
x = []
y = []
for folder in os.listdir(directory):
    print(dt.datetime.now())
    print(folder)
    for file in os.listdir(directory+'/'+folder):
        print(folder,file)
        emo = emotions.get(file.split('-')[2])
        features = extract_features('/'.join([directory,folder,file]))
        x.append(features)
        y.append(emo)
print(dt.datetime.now())
print('Done')
x = pd.DataFrame(x)
y = pd.Series(y)

2021-07-12 08:24:35.194529
Actor_01
Actor_01 03-02-01-01-01-01-01.wav
Actor_01 03-02-01-01-01-02-01.wav
Actor_01 03-02-01-01-02-01-01.wav
Actor_01 03-02-01-01-02-02-01.wav
Actor_01 03-02-02-01-01-01-01.wav
Actor_01 03-02-02-01-01-02-01.wav
Actor_01 03-02-02-01-02-01-01.wav
Actor_01 03-02-02-01-02-02-01.wav
Actor_01 03-02-02-02-01-01-01.wav
Actor_01 03-02-02-02-01-02-01.wav
Actor_01 03-02-02-02-02-01-01.wav
Actor_01 03-02-02-02-02-02-01.wav
Actor_01 03-02-03-01-01-01-01.wav
Actor_01 03-02-03-01-01-02-01.wav
Actor_01 03-02-03-01-02-01-01.wav
Actor_01 03-02-03-01-02-02-01.wav
Actor_01 03-02-03-02-01-01-01.wav
Actor_01 03-02-03-02-01-02-01.wav
Actor_01 03-02-03-02-02-01-01.wav
Actor_01 03-02-03-02-02-02-01.wav
Actor_01 03-02-04-01-01-01-01.wav
Actor_01 03-02-04-01-01-02-01.wav
Actor_01 03-02-04-01-02-01-01.wav
Actor_01 03-02-04-01-02-02-01.wav
Actor_01 03-02-04-02-01-01-01.wav
Actor_01 03-02-04-02-01-02-01.wav
Actor_01 03-02-04-02-02-01-01.wav
Actor_01 03-02-04-02-02-02-01.wav
Actor_01 03-

IndexError: list index out of range

In [8]:
len(y)

1012

In [ ]:
form 

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.25, random_state=101)

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [13]:
model_dt = DecisionTreeClassifier().fit(xtrain,ytrain)
model_dt.score(xtest,ytest)

0.4980237154150198

In [14]:
model_rf = RandomForestClassifier().fit(xtrain,ytrain)
model_rf.score(xtest,ytest)

0.8023715415019763

In [15]:
model_svc = SVC().fit(xtrain,ytrain)
model_svc.score(xtest,ytest)

0.2964426877470356

In [17]:
model = MLPClassifier(batch_size=8,alpha=0.01,hidden_layer_sizes=(200,),max_iter=500,learning_rate='adaptive',verbose=True,early_stopping=False,n_iter_no_change=100).fit(xtrain,ytrain)

Iteration 1, loss = 26.55832382
Iteration 2, loss = 23.07911179
Iteration 3, loss = 21.71892056
Iteration 4, loss = 20.83256202
Iteration 5, loss = 21.28386963
Iteration 6, loss = 19.11137608
Iteration 7, loss = 18.71870545
Iteration 8, loss = 19.67552054
Iteration 9, loss = 19.16844613
Iteration 10, loss = 16.05534561
Iteration 11, loss = 17.16671978
Iteration 12, loss = 16.73972204
Iteration 13, loss = 15.52970057
Iteration 14, loss = 16.71808819
Iteration 15, loss = 15.02412732
Iteration 16, loss = 13.34177998
Iteration 17, loss = 16.23868624
Iteration 18, loss = 15.02418930
Iteration 19, loss = 13.90495747
Iteration 20, loss = 13.70285226
Iteration 21, loss = 14.35206353
Iteration 22, loss = 17.42068692
Iteration 23, loss = 14.28861950
Iteration 24, loss = 11.96981491
Iteration 25, loss = 17.23024753
Iteration 26, loss = 14.75968584
Iteration 27, loss = 12.75762803
Iteration 28, loss = 14.22382936
Iteration 29, loss = 14.67773847
Iteration 30, loss = 14.59298050
Iteration 31, loss 

C:\Users\Amit\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [18]:
ypred = model.predict(xtest)


In [19]:
round(100*model.score(xtest,ytest),2)


70.36

In [21]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(ytest,ypred))


              precision    recall  f1-score   support

       angry       0.68      1.00      0.81        41
        calm       0.95      0.76      0.85        51
     fearful       0.58      0.56      0.57        45
       happy       0.68      0.77      0.72        39
     neutral       0.56      0.68      0.61        22
         sad       0.74      0.51      0.60        55

    accuracy                           0.70       253
   macro avg       0.70      0.71      0.69       253
weighted avg       0.72      0.70      0.70       253



In [22]:
# Recording the Sound


In [24]:
import sounddevice as sd
from scipy.io.wavfile import write

In [37]:
duration = 5# second
sample_freq = 22050

In [38]:
recording = sd.rec(int(duration*sample_freq),sample_freq,1)
sd.wait()

In [39]:
recording

array([[ 0.0000000e+00],
       [ 0.0000000e+00],
       [-3.0517578e-05],
       ...,
       [-1.8615723e-03],
       [-1.7395020e-03],
       [-2.6245117e-03]], dtype=float32)

In [40]:
write('sample.wav',sample_freq,recording)

In [41]:
ipd.Audio('sample.wav')

In [42]:
sig,rate=alib.load('sample.wav')

In [43]:
model.predict([extract_features('sample.wav')])

array(['calm'], dtype='<U7')